In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import our HMM models
import sys
sys.path.append('../../')
from hidden_regime.models.base_hmm import HiddenMarkovModel
from hidden_regime.models.online_hmm import OnlineHMM, OnlineHMMConfig
from hidden_regime.data.loader import DataLoader

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)


NameError: name 'List' is not defined